In [ ]:
import pandas as pd
import wandb
class Results():
    def __init__(self):
        num_seeds = 5
        columns_nguyen = [f"nguyen_{i}" for i in range(1, 13, 1)]
        columns_self = [f"self_{i}" for i in range(0, 10, 1)]
        columns = columns_nguyen.append(columns_self)
        self.result_dict = {}
        for target in ['num_states_to_perfect_fit_test', 'time to perfect fit', 'num simulation to perfect fit']:
            self.result_dict[target] = {}
            self.result_dict[target]['results_grammar_1_Normal'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_uniform_1_Normal'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_grammar_1_AmEx'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_uniform_1_AmEx'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)

            self.result_dict[target]['results_grammar_2_Normal'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_uniform_2_Normal'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_grammar_2_AmEx'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_uniform_2_AmEx'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            
            self.result_dict[target]['results_nn_1'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_nn_5'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_nn_15'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_nn_30'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_nn_45'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_nn_60'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)

    def fill_results(self, config, summary):
        seed = config['seed']
        data_path = config['data_path']
        engine = config['MCTS_engine']
        prior = config['prior_source']
        dataset = config['data_path'].split('/')[1]
        for target in ['num_states_to_perfect_fit_test', 'time to perfect fit', 'num simulation to perfect fit']:
            value = summary[target]
            if 'data_grammar_1' in data_path and engine == 'Endgame' and prior == 'grammar':
                self.result_dict[target]['results_grammar_1_AmEx'].loc[seed, dataset] = value
            elif 'data_grammar_1' in data_path and engine == 'Normal' and prior == 'grammar':
                self.result_dict[target]['results_grammar_1_Normal'].loc[seed, dataset] = value
            elif 'data_grammar_1' in data_path and engine == 'Endgame' and prior == 'uniform':
                self.result_dict[target]['results_uniform_1_AmEx'].loc[seed, dataset] = value
            elif 'data_grammar_1' in data_path and engine == 'Normal' and prior == 'uniform':
                self.result_dict[target]['results_uniform_1_Normal'].loc[seed, dataset] = value

            elif 'data_grammar_2' in data_path and engine == 'Endgame' and prior == 'grammar':
                self.result_dict[target]['results_grammar_2_AmEx'].loc[seed, dataset] = value
            elif 'data_grammar_2' in data_path and engine == 'Normal' and prior == 'grammar':
                self.result_dict[target]['results_grammar_2_Normal'].loc[seed, dataset] = value
            elif 'data_grammar_2' in data_path and engine == 'Endgame' and prior == 'uniform':
                self.result_dict[target]['results_uniform_2_AmEx'].loc[seed, dataset] = value
            elif 'data_grammar_2' in data_path and engine == 'Normal' and prior == 'uniform':
                self.result_dict[target]['results_uniform_2_Normal'].loc[seed, dataset] = value


api = wandb.Api()
entity, project = "jgu-wandb", "neural_guided_symbolic_regression_final"
# Example: January 1st, 2024

# created_at (str): ISO timestamp when the run was started
runs = api.runs(entity + "/" + project, filters={"tags": {"$in": ["no_net"]}})

summary_list, config_list, name_list = [], [], []
results_obj = Results()

for run in runs:
    # .summary contains output keys/values for
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files
    result_dict = run.summary._json_dict
    try:
        summary_dict = run.summary._json_dict
        config_dict = {k: v for k, v in run.config.items() if not k.startswith("_")}
        results_obj.fill_results(config_dict, summary_dict)
    except Exception as e:
        pass#print(f"{e} their is a problem with {run.name} ")

print('end')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
columns_nguyen = [f"nguyen_{i}" for i in range(1, 2, 1)]
#columns_self = [f"self_{i}" for i in range(0, 10, 1)]
columns = columns_nguyen #+ columns_self
experiments = ['results_grammar_1_AmEx','results_uniform_1_AmEx']#, 'NN1', 'NN_25' ]
target = 'num_states_to_perfect_fit_test'
fig, axs = plt.subplots(len(columns), len(experiments))
for row, dataset in enumerate(columns):
    for col, experiment in enumerate(experiments):
        pass
        df = results_obj.result_dict[target][experiment]
        sns.violinplot(data=df,
                       y=dataset,
                       ax=axs[row, col]             
                       )
        axs[row,col].set_title(f'{experiment}_{dataset}')